In [27]:
from itertools import count

from langchain.docstore.base import Docstore
from langchain.retrievers import ElasticSearchBM25Retriever

import guidance
from datasets import load_dataset
from elasticsearch import Elasticsearch

In [2]:
from ipython_secrets import get_secret

OPENAI_API_KEY = get_secret("OPENAI_API_KEY")

In [3]:
guidance.llm = guidance.llms.OpenAI(model="text-davinci-003", api_key=OPENAI_API_KEY)

In [6]:
data = load_dataset("xyzNLP/nza-ct-zoning-codes-text")
data = data["train"].to_pandas().drop(columns=["embeddings"])
data

Found cached dataset parquet (/Users/maxdumas/.cache/huggingface/datasets/xyzNLP___parquet/xyzNLP--nza-ct-zoning-codes-text-579051b4b18688cb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 151.84it/s]


,Town,Page,Text
0,vernon,1,
1,vernon,2,"ZONING REGULATIONS\nTown of Vernon, Connecticu..."
2,vernon,3,SECTION 1 - GENERAL\n1.1\nThe following regula...
3,vernon,4,1.3\nThe boundaries of these zones are shown o...
4,vernon,5,1.3A\nProposed changes to the Zoning Map shall...
...,...,...,...
22083,newtown,266,APPENDIX E\nDESIGN ADVISORY BOARD ORDINANCE\nB...
22084,newtown,267,APPENDIX F\nList of Permitted Uses in Commerci...
22085,newtown,268,APPENDIX G\nList of Permitted Uses in Industri...
22086,newtown,269,APPENDIX H\nARTICLE VIII - SUPPLEMENTAL REGULA...


In [7]:
class ElasticSearchDocStore(Docstore):
    def __init__(self, index_name: str, texts: list[str], reset_index=True):
        if reset_index:
            es = Elasticsearch("http://localhost:9200")  # default client
            es.indices.delete(index=index_name, ignore=[400, 404])
        
        self.retriever = ElasticSearchBM25Retriever.create(
            elasticsearch_url="http://localhost:9200",
            index_name=index_name,
            k1=2.0,
            b=0.75,
        )
        self.retriever.add_texts(texts=texts)

    def search(self, query):
        return self.retriever.get_relevant_documents(query)

In [8]:
town = "ansonia"
docstore=ElasticSearchDocStore("ansonia", data.query("Town == @town")["Text"].to_list())

/opt/homebrew/Caskroom/mambaforge/base/envs/zoning/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [9]:
docstore.search("AAA Residence, minimum lot size")

/opt/homebrew/Caskroom/mambaforge/base/envs/zoning/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


[Document(page_content='SCHEDULE D - MAXIMUM DWELLING DENSITY\nEffective: April 25, 2003.\n52\nCELL (1, 1): \nZoning District\nCELL (1, 2): \nMinimum Lot Size (In\nSquare Feet)\nCELL (1, 3): \nMax. Residential Density (Dwelling Units\nPer Acre)\nCELL (2, 1): \nAAA\nCELL (2, 2): \n60,000\nCELL (2, 3): \n0.59\nCELL (3, 1): \nAA\nCELL (3, 2): \n30,000\nCELL (3, 3): \n1.00\nCELL (4, 1): \nA\nCELL (4, 2): \n12,500\nCELL (4, 3): \n2.40\nCELL (5, 1): \nB\nCELL (5, 2): \n7,500\nCELL (5, 3): \n4.00\n', metadata={}),
 Document(page_content='Section 720.18 - Age-Restricted Housing - Special Exception\n720.18.1 - General\nThe Commission may, of its own initiative or in response to an application hereunder,\ndesignate sites within "A" Residence zoning district as suitable for conversion to an age-\nrestricted residential community providing that:\na.\nThe site is no less than four and one-third (4.3) acres in size; and it is\nentirely location within an "A" Residence district;\nb.\nThe site is full

In [43]:
valid_answers = ['Action', 'Final Answer']
valid_tools = ['Search']

prompt_start_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instructions
Answer the following questions as best you can. You have access to the following tools:

Search: Useful for when you need to answer questions about zoning documents. The input is a keyword search to retrieve pages containing relevant information.

### Schema

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action, should be a question.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

### Examples

Question: What is the minimum lot size, in square feet, for a single family home in the first listed residential district of the town of Ansonia, Connecticut?
Thought: I need to find the list of all zoning districts in Ansonia, Connecticut.
Action: Search
Action Input: zoning district list, Ansonia, Connecticut
Observation: 'ARTICLE II - ESTALISHMENT OF DISTRICTS AND ZONING\nADMINISTRATION\nSection 205 Zoning Districts Established\nFor the purposes of these regulations, the City of Ansonia is hereby divided into the following\nclasses of districts:\nSection 210 Zoning Map\nThe boundaries of the districts specified in Section 205 are hereby established as shown on a\nmap entitled "Official Zoning Map, City of Ansonia", dated February 1990, including any\nspecial maps and boundary descriptions supplementary thereto, adopted and declared to be a part\nof this Ordinance. In addition, the boundaries of the Flood Plain District are as delineated in\nSection 220 - Flood Plain District.\nSection 215 - Determination of District Boundaries\nIn determining the boundaries of districts shown on the Official Zoning Map, the following rules\nshall apply\n215.1 Unless otherwise shown, the district boundaries shall be construed to coincide with\nthe center lines of streets, alleys, parkways, waterways and main tracks of railroads. -\n215.2\nWhere such boundaries are indicated as approximate following the property lines of\nparks or other publicly owned lands, such lines shall be construed to be such\n17\nCELL (1, 1): \nDistrict\nCELL (1, 2): \nMap Code\nCELL (2, 1): \nAAA Residence District\nCELL (2, 2): \nAAA\nCELL (3, 1): \nAA Residence District\nCELL (3, 2): \nAA\nCELL (4, 1): \nA Residence District\nCELL (4, 2): \nA\nCELL (5, 1): \nB Residence District\nCELL (5, 2): \nB\nCELL (6, 1): \nGA Multi-Family Residence District\nCELL (6, 2): \nGA\nCELL (7, 1): \nMM Multi-Family Residence District\nCELL (7, 2): \nMM\nCELL (8, 1): \nBB Multi-Family Residence District\nCELL (8, 2): \nBB\nCELL (9, 1): \nRR Multi-Family Residence-Retail District\nCELL (9, 2): \nRR\nCELL (10, 1): \nNR Neighborhood-Retail District\nCELL (10, 2): \nNR\nCELL (11, 1): \nC Central Commercial District\nCELL (11, 2): \nC\nCELL (12, 1): \nSC Special Commercial District\nCELL (12, 2): \nSC\nCELL (13, 1): \nLI Light Industrial District\nCELL (13, 2): \nLI\nCELL (14, 1): \nHI Heavy Industrial District\nCELL (14, 2): \nHI\nCELL (15, 1): \nFlood Plain District\nCELL (15, 2): \nCELL (16, 1): \nCP Commerce Park District\nCELL (16, 2): \nCP\nCELL (17, 1): \nCCZ City Center Overlay Zone\nCELL (17, 2): \nCCOZ\n'
Thought: The first residential district is AAA Residence District. I need to find the minimum lot size for a single family home in this district.
Action: Search
Action Input: AAA Residence District, minimum lot size, single family home
Observation: 'SCHEDULE D - MAXIMUM DWELLING DENSITY\nEffective: April 25, 2003.\n52\nCELL (1, 1): \nZoning District\nCELL (1, 2): \nMinimum Lot Size (In\nSquare Feet)\nCELL (1, 3): \nMax. Residential Density (Dwelling Units\nPer Acre)\nCELL (2, 1): \nAAA\nCELL (2, 2): \n60,000\nCELL (2, 3): \n0.59\nCELL (3, 1): \nAA\nCELL (3, 2): \n30,000\nCELL (3, 3): \n1.00\nCELL (4, 1): \nA\nCELL (4, 2): \n12,500\nCELL (4, 3): \n2.40\nCELL (5, 1): \nB\nCELL (5, 2): \n7,500\nCELL (5, 3): \n4.00\n'
Thought: The table indicates that the minimum lot size for a single family home in the AAA Residence District is 60,000 square feet. So the answer is 60,000 square feet.
Final Answer: 60000

### Input
{{question}}

### Response
Question: {{question}}
Thought: {{gen 'thought' stop='\n'}}
{{select 'answer' options=valid_answers}}: """

prompt_mid_template = """{{select 'tool_name' options=valid_tools}}
Action Input: {{gen 'actInput' stop='\n'}}
Observation: {{do_tool tool_name actInput}}
Thought: {{gen 'thought' stop='\n'}}
{{select 'answer' options=valid_answers}}: """

In [44]:
def searchDocuments(t):
    # TODO: Update this to accept an explicit index from the town name.
    return docstore.search(t)[0].page_content

tools_dict = {
    'Search': searchDocuments
}

class CustomAgentGuidance:
    def __init__(self, guidance, tools, num_iter=3):
        self.guidance = guidance
        self.tools = tools
        self.num_iter = num_iter

    def do_tool(self, tool_name, actInput):
        return self.tools[tool_name](actInput)
    
    def __call__(self, query):
        # Create initial prompt with question, examples, and first round of thought
        prompt_start = self.guidance(prompt_start_template)
        result_start = prompt_start(question=query, valid_answers=valid_answers)

        result_mid = result_start
        
        # Until we reach a final answer or we reach the max number of
        # iterations, keep reacting
        for iter in count():
            history = str(result_mid)
            match result_mid['answer']:
                case "Action" if iter < self.num_iter:
                    # Intermediate action
                    prompt_mid = self.guidance(history + prompt_mid_template)
                    result_mid = prompt_mid(do_tool=self.do_tool, valid_answers=valid_answers, valid_tools=valid_tools)
                case "Action" if iter >= self.num_iter:
                    # Reach iteration limit but did not get a final answer; force one
                    prompt_mid = self.guidance(history + prompt_mid_template + "{{gen 'fn' stop='\\n'}}")
                    result_final = prompt_mid(do_tool=self.do_tool, valid_answers=['Final Answer'], valid_tools=valid_tools)
                    return result_final['fn'].strip()
                case "Final Answer":
                    # Naturally arrived at final answer
                    prompt_mid = self.guidance(history + "{{gen 'fn' stop='\\n'}}")
                    result_final = prompt_mid()
                    return result_final['fn'].strip()
                case _:
                    raise ValueError(f"Invalid intermediate answer: {result_mid['answer']}")

custom_agent = CustomAgentGuidance(guidance, tools_dict, num_iter=3)

In [45]:
custom_agent('What is the minimum lot size, in square feet, building in the  district of the town of Bethel, Connecticut?')

Stop program Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instructions
Answer the following questions as best you can. You have access to the following tools:

Search: Useful for when you need to answer questions about zoning documents. The input is a keyword search to retrieve pages containing relevant information.

### Schema

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action, should be a question.
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

### Examples

Question: What is the minimum lot size, in square feet, for a single family home in the first listed residential district of the town of Ansonia, Connecticut?
Thought: I need to find the list of all zoning districts in Ansonia, Connecticut.
Action: Search
Action Input: zoning district list, Ansonia, Connecticut
Observation: 'ARTICLE II - ESTALISHMENT OF DISTRICTS AND ZONING
ADMINISTRATION
Section 205 Zoning Districts Established
For the purposes of these regulations, the City of Ansonia is hereby divided into the following
classes of districts:
Section 210 Zoning Map
The boundaries of the districts specified in Section 205 are hereby established as shown on a
map entitled "Official Zoning Map, City of Ansonia", dated February 1990, including any
special maps and boundary descriptions supplementary thereto, adopted and declared to be a part
of this Ordinance. In addition, the boundaries of the Flood Plain District are as delineated in
Section 220 - Flood Plain District.
Section 215 - Determination of District Boundaries
In determining the boundaries of districts shown on the Official Zoning Map, the following rules
shall apply
215.1 Unless otherwise shown, the district boundaries shall be construed to coincide with
the center lines of streets, alleys, parkways, waterways and main tracks of railroads. -
215.2
Where such boundaries are indicated as approximate following the property lines of
parks or other publicly owned lands, such lines shall be construed to be such
17
CELL (1, 1): 
District
CELL (1, 2): 
Map Code
CELL (2, 1): 
AAA Residence District
CELL (2, 2): 
AAA
CELL (3, 1): 
AA Residence District
CELL (3, 2): 
AA
CELL (4, 1): 
A Residence District
CELL (4, 2): 
A
CELL (5, 1): 
B Residence District
CELL (5, 2): 
B
CELL (6, 1): 
GA Multi-Family Residence District
CELL (6, 2): 
GA
CELL (7, 1): 
MM Multi-Family Residence District
CELL (7, 2): 
MM
CELL (8, 1): 
BB Multi-Family Residence District
CELL (8, 2): 
BB
CELL (9, 1): 
RR Multi-Family Residence-Retail District
CELL (9, 2): 
RR
CELL (10, 1): 
NR Neighborhood-Retail District
CELL (10, 2): 
NR
CELL (11, 1): 
C Central Commercial District
CELL (11, 2): 
C
CELL (12, 1): 
SC Special Commercial District
CELL (12, 2): 
SC
CELL (13, 1): 
LI Light Industrial District
CELL (13, 2): 
LI
CELL (14, 1): 
HI Heavy Industrial District
CELL (14, 2): 
HI
CELL (15, 1): 
Flood Plain District
CELL (15, 2): 
CELL (16, 1): 
CP Commerce Park District
CELL (16, 2): 
CP
CELL (17, 1): 
CCZ City Center Overlay Zone
CELL (17, 2): 
CCOZ
'
Thought: The first residential district is AAA Residence District. I need to find the minimum lot size for a single family home in this district.
Action: Search
Action Input: AAA Residence District, minimum lot size, single family home
Observation: 'SCHEDULE D - MAXIMUM DWELLING DENSITY
Effective: April 25, 2003.
52
CELL (1, 1): 
Zoning District
CELL (1, 2): 
Minimum Lot Size (In
Square Feet)
CELL (1, 3): 
Max. Residential Density (Dwelling Units
Per Acre)
CELL (2, 1): 
AAA
CELL (2, 2): 
60,000
CELL (2, 3): 
0.59
CELL (3, 1): 
AA
CELL (3, 2): 
30,000
CELL (3, 3): 
1.00
CELL (4, 1): 
A
CELL (4, 2): 
12

'30000'